In [1]:
# import libraries
### bugs for transformers(4.28.1), update to newest version 4.31.0
from dataclasses import dataclass, field
from typing import Optional
import torch
import re
import numpy as np
import torch
import random
from accelerate import Accelerator
from datasets import load_dataset

from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, HfArgumentParser, pipeline
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel

[2023-09-23 20:17:26,809] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


# Random Seed

In [2]:
def set_seed(seed: int):
    """
   A function to fix random seed in `numpy`,`random`,  and `torch`.

    Args:
        seed (`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
set_seed(1000)

# Metrics

In [3]:
import operator
from functools import reduce
from math import fabs
from random import shuffle

try:
    from scipy.stats.stats import betai
except ImportError:
    betai = None

from nltk.util import LazyConcatenation, LazyMap


def accuracy(reference, test):
    """
    Given a list of reference values and a corresponding list of test
    values, return the fraction of corresponding values that are
    equal.  In particular, return the fraction of indices
    ``0<i<=len(test)`` such that ``test[i] == reference[i]``.

    :type reference: list
    :param reference: An ordered list of reference values.
    :type test: list
    :param test: A list of values to compare against the corresponding
        reference values.
    :raise ValueError: If ``reference`` and ``length`` do not have the
        same length.
    """
    if len(reference) != len(test):
        raise ValueError("Lists must have the same length.")
    return sum(x == y for x, y in zip(reference, test)) / len(test)



def precision(reference, test):
    """
    Given a set of reference values and a set of test values, return
    the fraction of test values that appear in the reference set.
    In particular, return card(``reference`` intersection ``test``)/card(``test``).
    If ``test`` is empty, then return None.

    :type reference: set
    :param reference: A set of reference values.
    :type test: set
    :param test: A set of values to compare against the reference set.
    :rtype: float or None
    """
    if not hasattr(reference, "intersection") or not hasattr(test, "intersection"):
        raise TypeError("reference and test should be sets")

    if len(test) == 0:
        return None
    else:
        return len(reference.intersection(test)) / len(test)



def recall(reference, test):
    """
    Given a set of reference values and a set of test values, return
    the fraction of reference values that appear in the test set.
    In particular, return card(``reference`` intersection ``test``)/card(``reference``).
    If ``reference`` is empty, then return None.

    :type reference: set
    :param reference: A set of reference values.
    :type test: set
    :param test: A set of values to compare against the reference set.
    :rtype: float or None
    """
    if not hasattr(reference, "intersection") or not hasattr(test, "intersection"):
        raise TypeError("reference and test should be sets")

    if len(reference) == 0:
        return None
    else:
        return len(reference.intersection(test)) / len(reference)



def f_measure(reference, test, alpha=0.5):
    """
    Given a set of reference values and a set of test values, return
    the f-measure of the test values, when compared against the
    reference values.  The f-measure is the harmonic mean of the
    ``precision`` and ``recall``, weighted by ``alpha``.  In particular,
    given the precision *p* and recall *r* defined by:

    - *p* = card(``reference`` intersection ``test``)/card(``test``)
    - *r* = card(``reference`` intersection ``test``)/card(``reference``)

    The f-measure is:

    - *1/(alpha/p + (1-alpha)/r)*

    If either ``reference`` or ``test`` is empty, then ``f_measure``
    returns None.

    :type reference: set
    :param reference: A set of reference values.
    :type test: set
    :param test: A set of values to compare against the reference set.
    :rtype: float or None
    """
    p = precision(reference, test)
    r = recall(reference, test)
    if p is None or r is None:
        return None
    if p == 0 or r == 0:
        return 0
    return 1.0 / (alpha / p + (1 - alpha) / r)


# few shot Prompt format

In [6]:
cot_examples_1 = """### Instruction:
Given a question, generate some helpful and creative thoughts step by step and then answer the question.

### Examples:

Question: 
There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
Thought:
There are 15 trees originally.
Then there were 21 trees after some more were planted.
So there must have been 21 - 15 = 6.
#Answer: 6

Question: 
If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
Thought:
There are originally 3 cars.
2 more cars arrive.
3 + 2 = 5.
#Answer: 5

Question: 
Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
Thought:
Originally, Leah had 32 chocolates.
Her sister had 42.
So in total they had 32 + 42 = 74.
After eating 35, they had 74 - 35 = 39.
#Answer: 39
"""
cot_examples_2="""
Choose the solution to achieve the goal and give answer in bracket:
### goal: When boiling butter, when it's ready, you can: 
### sol1: Pour it onto a plate
### sol2: Pour it into a jar
### Thought: When boiling butter, if it's ready, it is often recommended to use Solution 2: Pour it into a jar. Pouring the boiled butter into a jar allows it to be stored more easily and conveniently, keeping it fresh for later use. Pouring it onto a plate might not be as practical for storage and might lead to quicker spoilage.
### Therefore, the answer is sol [1]

Choose the solution to achieve the goal and give answer in bracket:
### goal: When boiling butter, when it's ready, you can: 
### sol1: Pour it onto a plate
### sol2: Pour it into a jar
### Thought: When boiling butter, if it's ready, it is often recommended to use Solution 2: Pour it into a jar. Pouring the boiled butter into a jar allows it to be stored more easily and conveniently, keeping it fresh for later use. Pouring it onto a plate might not be as practical for storage and might lead to quicker spoilage.
### Therefore, the answer is sol [2]

Choose the solution to achieve the goal and give answer in bracket:
### goal: how do you indent something?: 
### sol1: leave a space before starting the writing
### sol2: press the spacebar
### Thought: Indentation in writing is commonly achieved to visually separate or format content. It is typically done by starting the line with a certain number of spaces or using the "Tab" key on the keyboard. This helps improve readability and organization, especially in paragraphs or programming code.
### Therefore, the answer is sol [1]
"""

# Dataset

In [7]:
def build_test_dataset(
    dataset_name="/root/autodl-tmp/SVAMP",prompt_examples=cot_examples_1,prompt_format='default'
):
    
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.
        prompt_examples ('str'):
            The few shot examples for testing
        prompt_format ('str'):
            The format of the prompting for testing

    Returns:
        dataset for test of checkpoints of llms
    """


    ds_test = load_dataset(dataset_name, split="test")
    original_columns = ds_test.column_names


    def preprocess_function(examples):

        body=examples["Body"]
        question=examples["Question"]
        if prompt_format=='default':
            query = prompt_examples +'\n\n### Input:\n'+ 'Question:\n' + body +'\n'+ question + '\n\n' + "### Response:\n"

        return {"query": query }
    

    ds_test = ds_test.map(
        preprocess_function,
        batched=False,
        #remove_columns=original_columns,
    )


    print(ds_test)
    return ds_test

In [8]:
# We retrieve the dataloader by calling the `build_dataset` function.
dataset = build_test_dataset(dataset_name="/root/autodl-tmp/SVAMP")
prompt_test=dataset["query"]
answer_test=dataset["Answer"]

Found cached dataset json (/root/.cache/huggingface/datasets/json/SVAMP-ba92263a9f37e5fb/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
Loading cached processed dataset at /root/.cache/huggingface/datasets/json/SVAMP-ba92263a9f37e5fb/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-97ea55e108ce1fa3.arrow


Dataset({
    features: ['Type', 'Body', 'ID', 'Question', 'Answer', 'Equation', 'query'],
    num_rows: 300
})


# Model and Generator

In [15]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Create the pipeline for new model
new_generator= pipeline("text-generation", model=new_model, tokenizer=tokenizer,
                         max_new_tokens=512,
                         #top_k=50,
                         temperature=1.0,
                         #top_p=0.95,
                         do_sample=False,
                         #repetition_penalty=1.1,
                         device=0,)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


## base model

In [9]:
# load base model
model_name='/root/autodl-tmp/Llama-2-7b-chat-hf'
base_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                  torch_dtype=torch.float16,
                                                  device_map=0,
                                                 )
# Create the pipeline for base model
base_generator= pipeline("text-generation", model=base_model, tokenizer=tokenizer,
                         max_new_tokens=256,
                         top_k=50,
                         temperature=1.0,
                         top_p=0.95,
                         do_sample=False,
                         #repetition_penalty=1.1,
                         device=0,)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## fine-tuned model

In [9]:
# load fine-tuned model
model_name='/root/autodl-tmp/Llama-2-7b-chat-hf'
new_model_name="/root/autodl-tmp/msc_ml/llama_2/ckpts_svamp/checkpoint_1000"
new_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                 torch_dtype=torch.float16,
                                                 device_map=0,
                                                )

new_model = PeftModel.from_pretrained(new_model, new_model_name)
new_model = new_model.merge_and_unload()

#model.add_adapter(lora_config, adapter_name="adapter_1")


# Create the pipeline for new model
new_generator= pipeline("text2text-generation", model=new_model, tokenizer=tokenizer,
                         max_new_tokens=256,
                         top_k=50,
                         temperature=1.0,
                         top_p=0.95,
                         do_sample=True,
                         repetition_penalty=1.1,
                         device=0,)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

NameError: name 'tokenizer' is not defined

# Generate responses to questions

In [13]:
"""
def generate_responses(model,tokenizer,prompt_test=prompt_test,answer_test=answer_test,test_epochs=1,batch_size=2):


    # because of sampling of tokens, we can increase epochs of test for more precise evaluation
    test_epochs=test_epochs

    outputs_text=[]
    # the prompt and answer correspondong to output_text
    prompts_for_outputs=[]
    answers_for_outputs=[]

    batch_size = batch_size  

    for epoch in tqdm(range(test_epochs)):
        for i in tqdm(range(0, len(prompt_test), batch_size)):  # Increment by batch_size

            # Slice the data to create a batch
            batch_questions = prompt_test[i:i+batch_size]
            batch_answers = answer_test[i:i+batch_size]

            batch_inputs = tokenizer(batch_questions, padding=True, return_tensors="pt", truncation=True, max_length=2048).to('cuda')

            batch_outputs = model.generate(**batch_inputs,    
                                     max_new_tokens=256,
                                     top_k=50,
                                     temperature=1.0,
                                     top_p=0.95,
                                     do_sample=True,
                                     repetition_penalty=1.1)
            batch_input_length = batch_inputs.input_ids.shape[1]
            batch_generated_tokens = batch_outputs[:, batch_input_length:]
            batch_output_text = tokenizer.batch_decode(batch_generated_tokens)
            print(batch_output_text)
        
        
            outputs_text.extend(batch_output_text)
            prompts_for_outputs.extend(batch_questions)
            answers_for_outputs.extend(batch_answers)
    return outputs


outputs=generate_responses(model=base_model,tokenizer=tokenizer)
"""

In [11]:
def generate_responses(generator,prompt_test=prompt_test,answer_test=answer_test,test_epochs=1,batch_size=4):
    
    generator=generator

    # because of sampling of tokens, we can increase epochs of test for more precise evaluation
    test_epochs=test_epochs

    outputs=[]
    prompts_for_outputs=[]
    answers_for_outputs=[]

    batch_size = batch_size  

    for epoch in tqdm(range(test_epochs)):
        for i in tqdm(range(0, len(prompt_test), batch_size)):  # Increment by batch_size

            # Slice the data to create a batch
            batch_questions = prompt_test[i:i+batch_size]
            batch_answers = answer_test[i:i+batch_size]

            batch_generations = generator(batch_questions,batch_size=batch_size,return_full_text=False)
            print(batch_generations)
            outputs.extend([gen[0]['generated_text'] for gen in batch_generations])
            prompts_for_outputs.extend(batch_questions)
            answers_for_outputs.extend(batch_answers)
    return outputs,answers_for_outputs

In [16]:
responses,response_answers=generate_responses(new_generator)

  1%|          | 1/150 [00:20<49:55, 20.11s/it]

[[{'generated_text': 'Thought:\nThere are 6 cups of flour, 8 cups of sugar, and 7 cups of salt in the recipe.\nMary already put in 5 cups of flour.\nSo there are 6 - 5 = 1 cup of flour left to add.\nThere are 8 - 5 = 3 cups of sugar left to add.\nAnd there are 7 - 5 = 2 cups of salt left to add.\n#Answer: 3 cups of sugar and 2 cups of salt.'}], [{'generated_text': '\nThought:\nPaul had 492 crayons left.\nHe gave away 52 crayons.\nSo he lost 535 - 52 = 483 crayons.\n#Answer: 483\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n


  1%|▏         | 2/150 [00:39<49:13, 19.95s/it]

[[{'generated_text': "\nThought:\nOriginally, Robin's hair was 16 inches long.\nThen he cut off 11 inches.\nSo his hair was 16 - 11 = 5 inches long.\nThen it grew by 12 inches.\n5 + 12 = 17 inches.\n#Answer: 17 inches\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\


  2%|▏         | 3/150 [00:59<48:46, 19.90s/it]

...

[[{'generated_text': '\nThought:\nDebby drank 109 bottles a day.\nSo she drank 109 x 74 = 8060 bottles in total.\n#Answer: 8060\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\


 31%|███▏      | 47/150 [13:11<29:33, 17.22s/it]

[[{'generated_text': 'Thought:\nThe grasshopper jumped 13 inches.\nThe grasshopper jumped 2 inches farther than the grasshopper, so the frog jumped 13 + 2 = 15 inches.\n#Answer: 15\n\n\n### Instruction:\nGiven a question, generate some helpful and creative thoughts step by step and then answer the question.\n\n### Examples:\n\nQuestion: \nA bookshelf has 10 books on it. If 3 more books are added, how many books are on the bookshelf now?\nThought:\nThere are originally 10 books on the bookshelf.\n3 more books are added.\n10 + 3 = 13.\n#Answer: 13\n\nQuestion: \nA pizza has 8 slices. If 2 more slices are added, how many slices are on the pizza now?\nThought:\nThere are originally 8 slices on the pizza.\n2 more slices are added.\n8 + 2 = 10.\n#Answer: 10\n\nQuestion: \nA box has 5 toys inside. If 3 more toys are added, how many toys are in the box now?\nThought:\nThere are originally 5 toys inside the box.\n3 more toys are added.\n5 + 3 = 8.\n#Answer: 8\n\n\n### Input:\nQuestion:\nA water

  0%|          | 0/1 [13:24<?, ?it/s]


KeyboardInterrupt: 

# Process Responses

In [17]:
ANS_RE = re.compile(r"#Answer: (\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"
# extract the numerical answer for arithmetic dataset of the default prompt format
def _extract_answer(completion: str):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS

    
extracted_responses = [_extract_answer(response) for response in responses]

NameError: name 'responses' is not defined

# Record the results

In [ ]:
import pandas as pd



# Create a DataFrame with named columns
df_base_svamp = pd.DataFrame({'Question': response_questions, 'Response': responses,'extracted_response':extracted_responses, 'Answer': response_answers})

# Write to CSV
df_base_svamp.to_csv('test_base_svamp.csv', index=False)

# calculate metric

In [ ]:
svamp_base_accuracy=accuracy(reference=extracted_responses, test=response_answers)